## Modelling

### Import Neccesary Libraries

In [5]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

In [14]:
# Import custom helper functions
from helpers import make_windows,train_test_split,model_checkpoint_callback, evaluate_model, predict, plot_series

## Import data

In [ ]:
data = pd.read_csv('please enter path to prepared data')

In [ ]:
# Normalize our data
scaler = StandardScaler()
scaler = scaler.fit(data)
scaled_data = scaler.transform(data)

In [11]:
HORIZON = 5
WINDOW = 60

In [ ]:
windows, labels = make_windows(scaled_data)

### Split the window and label data into train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(windows, labels)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Convert our data to tensors
X_train = tf.constant(tf.cast(X_train, tf.float32))
y_train = tf.constant(tf.cast(y_train, tf.float32))

X_test = tf.constant(tf.cast(X_test, tf.float32))
y_test = tf.constant(tf.cast(y_test, tf.float32))

## Build our model

In [ ]:
# Create a random seed to make our model reproducible
tf.random.set_seed(42)

base_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(60, 2)),
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=False),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(2)
], name='base_model')

# compile our model
base_model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mae'])

# fit our model
history = base_model.fit(X_train, 
                        y_train, 
                        epochs=5, 
                        batch_size=60,
                        validation_data=(X_test, y_test),
                        shuffle=False,
                        callbacks=[model_checkpoint_callback('BaseModel')])

# summarize our model
base_model.summary()

### Evaluate our model performance from the best saved checkpoint

In [ ]:
evaluate_model('Callbacks/checkpoints/', 
               x_test=X_test, 
               y_test=y_test)

## Make a forecast
**📝 Note:** These are only psuedo forecasts

In [ ]:
y_pred = predict(base_model, X_test)